# FLIP(01):  Advanced Data Science
**(Tools Module 04: TPOP)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use, but NOT allowed to change or distribute this package.

Prepared by and for 
**Student Members** |
2006-2018 [TULIP Lab](http://www.tulip.org.au)

---


# Session 02 - Scoring Functions

TPOT makes use of `sklearn.model_selection.cross_val_score` for evaluating pipelines, and as such offers the same support for scoring functions. There are two ways to make use of scoring functions with TPOT:

* You can pass in a string to the `scoring` parameter from the list above. Any other strings will cause TPOT to throw an exception.

* You can pass the callable object/function with signature `scorer(estimator, X, y)`, where `estimator` is trained estimator to use for scoring, `X` are features that will be passed to `estimator.predict` and `y` are target values for `X`. To do this, you should implement your own function. See the example below for further explanation.

```Python from tpot import TPOTClassifier from sklearn.datasets import load_digits from sklearn.model_selection import train_test_split from sklearn.metrics.scorer import make_scorer

digits = load_digits() X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, train_size=0.75, test_size=0.25) 
* Make a custom metric function def my_custom_accuracy(y_true, y_pred): return float(sum(y_pred == y_true)) / len(y_true)

* Make a custom a scorer from the custom metric function # Note: greater_is_better=False in make_scorer below would mean that the scoring function should be minimized. my_custom_scorer = make_scorer(my_custom_accuracy, greater_is_better=True)
<br><br>
tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2, scoring=my_custom_scorer) tpot.fit(X_train, y_train) print(tpot.score(X_test, y_test)) tpot.export('tpot_mnist_pipeline.py') ```

* You can pass a metric function with the signature `score_func(y_true, y_pred)` (e.g. `my_custom_accuracy` in the example above), where `y_true` are the true target values and `y_pred` are the predicted target values from an estimator. To do this, you should implement your own function. See the example above for further explanation. TPOT assumes that any function with "error" or "loss" in the function name is meant to be minimized (`greater_is_better=False` in `make_scorer`), whereas any other functions will be maximized. This scoring type was deprecated in version 0.9.1 and will be removed in version 0.11.

* my_module.scorer_name: You can also use a custom `score_func(y_true, y_pred)` or `scorer(estimator, X, y)` function through the command line by adding the argument `-scoring my_module.scorer` to your command-line call. TPOT will import your module and use the custom scoring function from there. TPOT will include your current working directory when importing the module, so you can place it in the same directory where you are going to run TPOT. 
<br><br>
Example: `-scoring sklearn.metrics.auc` will use the function auc from sklearn.metrics module.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator

"""
# NOTE: Make sure that the class is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1).values
training_features, testing_features, training_target, testing_target = \
    train_test_split(features, tpot_data['target'].values, random_state=None)
"""

# 以上代码需修改为下面形式以供正确运行
from sklearn.datasets import load_digits

digits = load_digits()
X = digits.data
y = digits.target

training_features, testing_features, training_target, testing_target = \
    train_test_split(X, y, random_state=None)

# 此为由TPOT遗传算法得到的最优模型及参数组合
# Average CV score on the training set was:0.9792963424938936

exported_pipeline = make_pipeline(    PolynomialFeatures(degree=2, include_bias=False, interaction_only=False),    ZeroCount(),    LinearSVC(C=0.5, dual=True, loss="squared_hinge", penalty="l2", tol=0.001))
exported_pipeline.fit(training_features, training_target) results = exported_pipeline.predict(testing_features) print(results)